## 변수는 다음과 같이 변환하다.
- cloud dummy추가
- 화씨전환
- month dummy 추가 -> good!!
- WindDirection은 제거해준다(plot근거로)


## 각 시간별 지역별 모델은 crossvalidation기준으로 선정
### us
- 8,17,19 -> RF   /   18 -> lgb       /  나머지 ->  LR

### dj_or
- 8,17,19 -> RF      /       18 -> lgb      /       나머지 ->  LR


### dj_wh
- 8,18,19 -> RF           /              나머지 ->  LR

### dj_ft
- 17,19 -> RF         /          18 ->  lgb         /        나머지 ->LR


## train 및 test 생성

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from datetime import timedelta

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

In [2]:
fcst_us=pd.read_csv("ulsan_fcst_best.csv")
fcst_dj=pd.read_csv("DJ_fcst_best.csv")
energy=pd.read_csv('energy.csv')

In [3]:
# 시간으로 분류를해본다

energy['date'] = energy['time'].apply(lambda x: x.split()[0])
energy['time'] = energy['time'].apply(lambda x: x.split()[1])
energy['time'] = energy['time'].str.rjust(8,'0') # 한자릿수 시간 앞에 0 추가 ex) 3시 -> 03시

# 24시를 00시로 바꿔주기
energy.loc[energy['time']=='24:00:00','time'] = '00:00:00'
energy['time'] = energy['date'] + ' ' + energy['time']
energy['time'] = pd.to_datetime(energy['time'])
energy.loc[energy['time'].dt.hour==0,'time'] += timedelta(days=1)



energy['year']=energy['time'].dt.year
energy['month']=energy['time'].dt.month
energy['day']=energy['time'].dt.day
energy['hour']=energy['time'].dt.hour

energy['day_int']=energy['year']*10000+energy['month']*100+energy['day']
energy['day_hour_int']=energy['day_int']*100+energy['hour']

energy


,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan,date,year,month,day,hour,day_int,day_hour_int
0,2018-03-01 01:00:00,0.0,0.0,0,0,2018-03-01,2018,3,1,1,20180301,2018030101
1,2018-03-01 02:00:00,0.0,0.0,0,0,2018-03-01,2018,3,1,2,20180301,2018030102
2,2018-03-01 03:00:00,0.0,0.0,0,0,2018-03-01,2018,3,1,3,20180301,2018030103
3,2018-03-01 04:00:00,0.0,0.0,0,0,2018-03-01,2018,3,1,4,20180301,2018030104
4,2018-03-01 05:00:00,0.0,0.0,0,0,2018-03-01,2018,3,1,5,20180301,2018030105
...,...,...,...,...,...,...,...,...,...,...,...,...
25627,2021-01-31 20:00:00,0.0,0.0,0,0,2021-01-31,2021,1,31,20,20210131,2021013120
25628,2021-01-31 21:00:00,0.0,0.0,0,0,2021-01-31,2021,1,31,21,20210131,2021013121
25629,2021-01-31 22:00:00,0.0,0.0,0,0,2021-01-31,2021,1,31,22,20210131,2021013122
25630,2021-01-31 23:00:00,0.0,0.0,0,0,2021-01-31,2021,1,31,23,20210131,2021013123


In [4]:
# 시간(hour) 변수 생성
fcst_dj['hour']= fcst_dj['Forecast_time'].str.split(' ').str[1].str.split(':').str[0].astype(int)
fcst_dj['date']= fcst_dj['Forecast_time'].str.split(' ').str[0]

# year montj day추가
year_fcst = fcst_dj['date'].str.split('-').str[0].astype(int)
month_fcst = fcst_dj['date'].str.split('-').str[1].astype(int)
day_fcst = fcst_dj['date'].str.split('-').str[2].astype(int)
fcst_dj['day_int']=year_fcst*10000+month_fcst*100+day_fcst

fcst_dj['year']=year_fcst
fcst_dj['month']=month_fcst
fcst_dj['day']=day_fcst

# 정렬을 쉽게하기위한 day_hour_int를 생성
fcst_dj['day_int']=year_fcst*10000+month_fcst*100+day_fcst
fcst_dj['day_hour_int']=fcst_dj['day_int']*100+fcst_dj['hour']

In [5]:
# 시간(hour) 변수 생성
fcst_us['hour']= fcst_us['Forecast_time'].str.split(' ').str[1].str.split(':').str[0].astype(int)
fcst_us['date']= fcst_us['Forecast_time'].str.split(' ').str[0]

# year montj day추가
year_fcst = fcst_us['date'].str.split('-').str[0].astype(int)
month_fcst = fcst_us['date'].str.split('-').str[1].astype(int)
day_fcst = fcst_us['date'].str.split('-').str[2].astype(int)

fcst_us['year']=year_fcst
fcst_us['month']=month_fcst
fcst_us['day']=day_fcst


# 정렬을 쉽게하기위한 day_hour_int를 생성
fcst_us['day_int']=year_fcst*10000+month_fcst*100+day_fcst
fcst_us['day_hour_int']=fcst_us['day_int']*100+fcst_us['hour']

# 변수변환

In [6]:
## 화씨 생성
fcst_dj['Temp_F']=fcst_dj['Temperature']*9/5+32
fcst_us['Temp_F']=fcst_us['Temperature']*9/5+32

#구름 더미화
fcst_incoded_dj_1=pd.get_dummies(fcst_dj,columns=['Cloud'])
fcst_incoded_us_1=pd.get_dummies(fcst_us,columns=['Cloud'])

fcst_incoded_dj=pd.get_dummies(fcst_incoded_dj_1,columns=['month'])
fcst_incoded_us=pd.get_dummies(fcst_incoded_us_1,columns=['month'])

In [7]:
fcst_incoded_dj

,Forecast_time,Temperature,Humidity,WindSpeed,hour,date,day_int,year,day,day_hour_int,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2018-03-01 15:00:00,0.000000,60.000000,7.300000,15,2018-03-01,20180301,2018,1,2018030115,...,1,0,0,0,0,0,0,0,0,0
1,2018-03-01 16:00:00,-0.923476,59.720137,7.233333,16,2018-03-01,20180301,2018,1,2018030116,...,1,0,0,0,0,0,0,0,0,0
2,2018-03-01 17:00:00,-1.590142,59.720137,7.166667,17,2018-03-01,20180301,2018,1,2018030117,...,1,0,0,0,0,0,0,0,0,0
3,2018-03-01 18:00:00,-2.000000,60.000000,7.100000,18,2018-03-01,20180301,2018,1,2018030118,...,1,0,0,0,0,0,0,0,0,0
4,2018-03-01 19:00:00,-2.153049,60.559726,7.233333,19,2018-03-01,20180301,2018,1,2018030119,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26309,2021-03-03 12:00:00,7.000000,40.000000,3.200000,12,2021-03-03,20210303,2021,3,2021030312,...,1,0,0,0,0,0,0,0,0,0
26310,2021-03-03 15:00:00,8.000000,40.000000,4.500000,15,2021-03-03,20210303,2021,3,2021030315,...,1,0,0,0,0,0,0,0,0,0
26311,2021-03-03 18:00:00,5.000000,55.000000,2.200000,18,2021-03-03,20210303,2021,3,2021030318,...,1,0,0,0,0,0,0,0,0,0
26312,2021-03-03 21:00:00,1.000000,80.000000,1.900000,21,2021-03-03,20210303,2021,3,2021030321,...,1,0,0,0,0,0,0,0,0,0


### 결합해서 데이터 생성

In [8]:
# train_X


train_us=fcst_incoded_us[(fcst_incoded_us['day_int']<=20210131)&(fcst_incoded_us['day_int']>=20180302)]
train_us=train_us.reset_index().drop(['index'],axis=1)

train_dj=fcst_incoded_dj[(fcst_incoded_dj['day_int']<=20210131)&(fcst_incoded_dj['day_int']>=20180302)]
train_dj=train_dj.reset_index().drop(['index'],axis=1)


train_dj_or=train_dj.copy()
train_dj_wh=train_dj.copy()
train_dj_ft=train_dj.copy()


#y값 추가 및 na 값 제거
train_y=energy[(energy['day_int']<=20210131)&(energy['day_int']>=20180302)]
train_y=train_y.reset_index().drop(['index'],axis=1)


train_us['energy']=train_y.ulsan
train_us=train_us.dropna(axis=0)

train_dj_or['energy']=train_y.dangjin
train_dj_or=train_dj_or.dropna(axis=0)

train_dj_wh['energy']=train_y.dangjin_warehouse
train_dj_wh=train_dj_wh.dropna(axis=0)

train_dj_ft['energy']=train_y.dangjin_floating
train_dj_ft=train_dj_ft.dropna(axis=0)





In [9]:
train_dj_or

,Forecast_time,Temperature,Humidity,WindSpeed,hour,date,day_int,year,day,day_hour_int,...,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,energy
0,2018-03-02 00:00:00,-2.000000,50.000000,5.800000,0,2018-03-02,20180302,2018,2,2018030200,...,0,0,0,0,0,0,0,0,0,0
1,2018-03-02 01:00:00,-2.230892,50.133445,5.533333,1,2018-03-02,20180302,2018,2,2018030201,...,0,0,0,0,0,0,0,0,0,0
2,2018-03-02 02:00:00,-2.587597,52.501622,5.266667,2,2018-03-02,20180302,2018,2,2018030202,...,0,0,0,0,0,0,0,0,0,0
3,2018-03-02 03:00:00,-3.000000,55.000000,5.000000,3,2018-03-02,20180302,2018,2,2018030203,...,0,0,0,0,0,0,0,0,0,0
4,2018-03-02 04:00:00,-3.456415,57.277824,4.700000,4,2018-03-02,20180302,2018,2,2018030204,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25603,2021-01-31 19:00:00,5.530108,78.288590,3.533333,19,2021-01-31,20210131,2021,31,2021013119,...,0,0,0,0,0,0,0,0,0,0
25604,2021-01-31 20:00:00,5.698114,79.587723,3.766667,20,2021-01-31,20210131,2021,31,2021013120,...,0,0,0,0,0,0,0,0,0,0
25605,2021-01-31 21:00:00,6.000000,80.000000,4.000000,21,2021-01-31,20210131,2021,31,2021013121,...,0,0,0,0,0,0,0,0,0,0
25606,2021-01-31 22:00:00,6.351762,79.709188,4.666667,22,2021-01-31,20210131,2021,31,2021013122,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# 각 train data 내보내기
train_us.to_csv("train_month_us.csv",index=False)
train_dj_or.to_csv("train_month_dj_or.csv",index=False)
train_dj_wh.to_csv("train_month_dj_wh.csv",index=False)
train_dj_ft.to_csv("train_month_dj_ft.csv",index=False)

## test 생성

In [11]:
test_us=fcst_incoded_us[(fcst_incoded_us['day_hour_int']>=2021020100)&(fcst_incoded_us['day_hour_int']<=2021022823)]
test_us=test_us.reset_index().drop(['index'],axis=1)
test_us['energy']=0

test_X_us=test_us[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12','day_hour_int','hour','energy']]



test_dj=fcst_incoded_dj[(fcst_incoded_dj['day_hour_int']>=2021020100)&(fcst_incoded_dj['day_hour_int']<=2021022823)]
test_dj=test_dj.reset_index().drop(['index'],axis=1)
test_dj['energy']=0

test_X_dj=test_dj[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12','day_hour_int','hour','energy']]


In [12]:
test_X_dj

,Humidity,WindSpeed,Temp_F,Cloud_1.0,Cloud_2.0,Cloud_3.0,Cloud_4.0,month_1,month_2,month_3,...,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_hour_int,hour,energy
0,80.000000,6.000000,44.600000,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,2021020100,0,0
1,82.900725,5.133333,44.846776,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,2021020101,1,0
2,87.243862,4.266667,44.852568,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,2021020102,2,0
3,90.000000,3.400000,44.600000,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,2021020103,3,0
4,90.664237,3.100000,44.086175,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,2021020104,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,58.242709,1.400000,44.625818,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,2021022819,19,0
668,59.813537,1.300000,43.480381,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,2021022820,20,0
669,60.000000,1.200000,42.800000,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,2021022821,21,0
670,58.850017,1.233333,42.557393,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,2021022822,22,0


## 변수선택 및 최적모형 예측

In [13]:

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsRegressor # k-최근접 이웃 회귀 #분류대신 값을 예측
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge




#개별 모델 성능 (MAE) 확인

knn_model = KNeighborsRegressor()
svr_model = SVR()
lr_model = LinearRegression()
rf_model = RandomForestRegressor()
lgb_model = LGBMRegressor()

In [29]:
PRED_us=pd.DataFrame(np.zeros((28,24)))
### 8,17,19 -> RF   /   18 -> lgb       /  나머지 ->  LR


for h in range(24):
########################  평가기준 아닌 시간은 0########################3    
    if h<=7:
        PRED_us.iloc[:,h]=0
    elif h>=20:
        PRED_us.iloc[:,h]=0
######################   18시 -> lgb   ####################################    
    elif h==18:      
        data=train_us[train_us['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_us[test_X_us['hour']==h].reset_index().drop(['index'],axis=1)


        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]
        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]



        lgb_model = LGBMRegressor()
        lgb_model.fit(train_X,train_y)

        #예측값
        energy=lgb_model.predict(test)
        PRED_us.iloc[:,h]=energy
#######################  8시 ->  RF ############################        
    elif h==8:
        data=train_us[train_us['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_us[test_X_us['hour']==h].reset_index().drop(['index'],axis=1)


        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]
        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]



        rf_model = RandomForestRegressor()
        rf_model.fit(train_X,train_y)

        #예측값
        energy=rf_model.predict(test)
        PRED_us.iloc[:,h]=energy
#######################  17시 ->  RF ############################        

    elif h>=17:
        data=train_us[train_us['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_us[test_X_us['hour']==h].reset_index().drop(['index'],axis=1)


        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]
        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]



        rf_model = RandomForestRegressor()
        rf_model.fit(train_X,train_y)

        #예측값
        energy=rf_model.predict(test)
        PRED_us.iloc[:,h]=energy
        
        
####################### 나머지-> LR ###############################
    else:
        data=train_us[train_us['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_us[test_X_us['hour']==h].reset_index().drop(['index'],axis=1)


        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]
        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]



        lr_model = LinearRegression()
        lr_model.fit(train_X,train_y)

        #예측값
        energy=lr_model.predict(test)
        PRED_us.iloc[:,h]=energy

In [30]:
PRED_dj_or=pd.DataFrame(np.zeros((28,24)))
# 8,17,19 -> RF      /       18 -> lgb      /       나머지 ->  LR


for h in range(24):
    if h<=7:
        PRED_dj_or.iloc[:,h]=0
    elif h>=20:
        PRED_dj_or.iloc[:,h]=0
######################   18시 -> lgb   ####################################    
    elif h==18:      
        data=train_dj_or[train_dj_or['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_dj[test_X_dj['hour']==h].reset_index().drop(['index'],axis=1)


        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]
        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]



        lgb_model = LGBMRegressor()
        lgb_model.fit(train_X,train_y)

        #예측값
        energy=lgb_model.predict(test)
        PRED_dj_or.iloc[:,h]=energy
#######################  8시 ->  RF ############################        
    elif h==8:
        data=train_dj_or[train_dj_or['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_dj[test_X_dj['hour']==h].reset_index().drop(['index'],axis=1)


        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]
        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]



        rf_model = RandomForestRegressor()
        rf_model.fit(train_X,train_y)

        #예측값
        energy=rf_model.predict(test)
        PRED_dj_or.iloc[:,h]=energy
#######################  17시,19시 ->  RF ############################        

    elif h>=17:
        data=train_dj_or[train_dj_or['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_dj[test_X_dj['hour']==h].reset_index().drop(['index'],axis=1)


        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]
        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]



        rf_model = RandomForestRegressor()
        rf_model.fit(train_X,train_y)

        #예측값
        energy=rf_model.predict(test)
        PRED_dj_or.iloc[:,h]=energy
         
####################### 나머지-> LR ###############################
    
    else:
        data=train_dj_or[train_dj_or['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_dj[test_X_dj['hour']==h].reset_index().drop(['index'],axis=1)



        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]
        
        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]


        lr_model = LinearRegression()
        lr_model.fit(train_X,train_y)

        #예측값
        energy=lr_model.predict(test)
        PRED_dj_or.iloc[:,h]=energy

In [28]:
PRED_dj_wh=pd.DataFrame(np.zeros((28,24)))
#    8,18,19 -> RF       /       나머지 ->  LR


for h in range(24):
    if h<=7:
        PRED_dj_wh.iloc[:,h]=0
    elif h>=20:
        PRED_dj_wh.iloc[:,h]=0
#######################  8시 ->  RF ############################        
    elif h==8:
        data=train_dj_wh[train_dj_wh['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_dj[test_X_dj['hour']==h].reset_index().drop(['index'],axis=1)


        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]
        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]



        rf_model = RandomForestRegressor()
        rf_model.fit(train_X,train_y)

        #예측값
        energy=rf_model.predict(test)
        PRED_dj_wh.iloc[:,h]=energy
#######################  18시,19시 ->  RF ############################        

    elif h>=18:
        data=train_dj_wh[train_dj_wh['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_dj[test_X_dj['hour']==h].reset_index().drop(['index'],axis=1)


        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]
        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]



        rf_model = RandomForestRegressor()
        rf_model.fit(train_X,train_y)

        #예측값
        energy=rf_model.predict(test)
        PRED_dj_wh.iloc[:,h]=energy
         
####################### 나머지-> LR ###############################
    else:
        data=train_dj_wh[train_dj_wh['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_dj[test_X_dj['hour']==h].reset_index().drop(['index'],axis=1)



        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]

        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]


        lr_model = LinearRegression()
        lr_model.fit(train_X,train_y)

        #예측값
        energy=lr_model.predict(test)
        PRED_dj_wh.iloc[:,h]=energy

In [32]:
PRED_dj_ft=pd.DataFrame(np.zeros((28,24)))
# 17,19 -> RF         /          18 ->  lgb         /        나머지 ->LR


for h in range(24):
    if h<=7:
        PRED_dj_ft.iloc[:,h]=0
    elif h>=20:
        PRED_dj_ft.iloc[:,h]=0
######################   18시 -> lgb   ####################################    
    elif h==18:      
        data=train_dj_ft[train_dj_ft['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_dj[test_X_dj['hour']==h].reset_index().drop(['index'],axis=1)


        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]
        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]



        lgb_model = LGBMRegressor()
        lgb_model.fit(train_X,train_y)

        #예측값
        energy=lgb_model.predict(test)
        PRED_dj_ft.iloc[:,h]=energy

#######################  17시,19시 ->  RF ############################        

    elif h>=17:
        data=train_dj_ft[train_dj_ft['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_dj[test_X_dj['hour']==h].reset_index().drop(['index'],axis=1)


        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]
        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]



        rf_model = RandomForestRegressor()
        rf_model.fit(train_X,train_y)

        #예측값
        energy=rf_model.predict(test)
        PRED_dj_ft.iloc[:,h]=energy
         
####################### 나머지-> LR ###############################
    else:
        data=train_dj_ft[train_dj_ft['hour']==h].reset_index().drop(['index'],axis=1)
        data=data[data['energy']>0]

        test_h=test_X_dj[test_X_dj['hour']==h].reset_index().drop(['index'],axis=1)



        train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']]

        #train_X=data[['Humidity','WindSpeed','Temp_F','Cloud_1.0','Cloud_2.0','Cloud_3.0','Cloud_4.0']]

        train_y=data.energy.reset_index().drop(['index'],axis=1)
        test=test_h.iloc[:,:-3]



        lr_model = LinearRegression()
        lr_model.fit(train_X,train_y)

        #예측값
        energy=lr_model.predict(test)
        PRED_dj_ft.iloc[:,h]=energy

In [33]:
PRED_np_us=PRED_us.to_numpy()
PRED_np_dj_or=PRED_dj_or.to_numpy()
PRED_np_dj_ft=PRED_dj_ft.to_numpy()
PRED_np_dj_wh=PRED_dj_wh.to_numpy()


In [34]:
test_X_dj['dangin']=PRED_np_dj_or.reshape(-1,1)
test_X_dj['dangin_warehouse']=PRED_np_dj_wh.reshape(-1,1)
test_X_dj['dangin_floating']=PRED_np_dj_ft.reshape(-1,1)
test_X_us['ulsan']=PRED_np_us.reshape(-1,1)

In [35]:
sub=pd.read_csv('sample_submission.csv')
sub_2=sub.iloc[:671,:]
sub_2
sub_67=sub.iloc[671:,:]
sub.head(10)

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0,0,0,0
1,2021-02-01 02:00:00,0,0,0,0
2,2021-02-01 03:00:00,0,0,0,0
3,2021-02-01 04:00:00,0,0,0,0
4,2021-02-01 05:00:00,0,0,0,0
5,2021-02-01 06:00:00,0,0,0,0
6,2021-02-01 07:00:00,0,0,0,0
7,2021-02-01 08:00:00,0,0,0,0
8,2021-02-01 09:00:00,0,0,0,0
9,2021-02-01 10:00:00,0,0,0,0


In [36]:
sub_2['dangjin']=list(test_X_dj.iloc[1:,:].dangin)
sub_2['dangjin_warehouse']=list(test_X_dj.iloc[1:,:].dangin_warehouse)
sub_2['dangjin_floating']=list(test_X_dj.iloc[1:,:].dangin_floating)
sub_2['ulsan']=list(test_X_us.iloc[1:,:].ulsan)



sub_final=pd.concat([sub_2,sub_67],axis=0)
sub_final.to_csv('sub_month_nan0_time_best.csv',index=False)

In [37]:
sub_final.shape

(1392, 5)

In [38]:
sub_final.head(10)

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0.000000,0.000000,0.000000,0.000000
1,2021-02-01 02:00:00,0.000000,0.000000,0.000000,0.000000
2,2021-02-01 03:00:00,0.000000,0.000000,0.000000,0.000000
3,2021-02-01 04:00:00,0.000000,0.000000,0.000000,0.000000
4,2021-02-01 05:00:00,0.000000,0.000000,0.000000,0.000000
5,2021-02-01 06:00:00,0.000000,0.000000,0.000000,0.000000
6,2021-02-01 07:00:00,0.000000,0.000000,0.000000,0.000000
7,2021-02-01 08:00:00,-11.542171,-14.346289,14.010000,5.760000
8,2021-02-01 09:00:00,2.878167,-8.612153,-5.676750,0.138709
9,2021-02-01 10:00:00,58.272285,16.166361,53.247875,39.580776
